In [1]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns
from sklearn.linear_model import LinearRegression

import requests
import csv
import json

import warnings

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

# PE TTM valuation

In [7]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Custmized
ticker_symbols = [

    'CL'
    ,'ACN'
    # ,'GOOG'
    # ,'TSM'
    # ,'META'
    # ,'LPG'
    # ,'NVDA'
    # ,'SKWD'
    # ,'MSFT'
    ,'NFLX'
    # ,'V'
    ,'PG'
    ]

# ticker_symbols = [
#  'MO',
#  'ADM',
# #  'BF.B',
#  'BG',
#  'CPB',
#  'CHD',
#  'CLX',
#  'KO',
#  'CL',
#  'CAG',
#  'STZ',
#  'COST',
#  'DG',
#  'DLTR',
#  'EL',
#  'GIS',
#  'HSY',
#  'HRL',
# #  'KVUE',
#  'KDP',
#  'KMB',
#  'KHC',
#  'KR',
#  'LW',
#  'MKC',
#  'TAP',
#  'MDLZ',
#  'MNST',
#  'PEP',
#  'PM',
#  'PG',
#  'SJM',
#  'SYY',
#  'TGT',
#  'TSN',
# #  'WBA',
#  'WMT'
#  ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 9 # this will return x-1 yr PE range, cuz you cannot get current year end PE

ticker_dict_json = {}
ticker_dict_pd = {}

In [8]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

In [9]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })
                # stock_split_record_df = pd.DataFrame()
                # stock_split_record_df['split_factor'] = 1
                # stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API -- for now we use yf API
    # # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    # CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    # with requests.Session() as s:
    #     download = s.get(CSV_URL)
    #     decoded_content = download.content.decode('utf-8')
    #     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    #     my_list = list(cr)

    #     forecast_earanings_df = pd.DataFrame(
    #         columns=my_list[0]
    #         ,data=my_list[1::]
    #         )
        
    #     if forecast_earanings_df['estimate'].head(1).values != '':
    #         latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
    #     else:
    #         latest_projected_EPS = 0


    # forecast 1 qtr earning, 1yr earning from yf API
    yf_symbol = yf.Ticker(symbol)
    stock_eps_forecast_df = yf_symbol.earnings_estimate
    stock_eps_forecast_df['period'] = stock_eps_forecast_df.index
    next_qtr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1q']['avg'].values[0]
    # next yr eps
    next_yr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1y']['avg'].values[0]


    # EPS trend from yf API
    stock_eps_trend_df = yf_symbol.eps_trend
    stock_eps_trend_df['period'] = stock_eps_trend_df.index
    next_yr_days7ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['7daysAgo'].values[0]
    next_yr_days30ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['30daysAgo'].values[0]
    next_yr_days60ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['60daysAgo'].values[0]
    next_yr_days90ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['90daysAgo'].values[0]



    # stock growth rate from yf API
    stock_growth_est_df = yf_symbol.growth_estimates
    stock_growth_est_df['period'] = stock_growth_est_df.index
    curr_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['stockTrend'].values[0] * 100
    next_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['stockTrend'].values[0] * 100
    curr_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['indexTrend'].values[0] * 100
    next_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['indexTrend'].values[0] * 100


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = next_qtr_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = next_qtr_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected
        stock_consolidate_df["EPS_nextYr"] = next_yr_EPS

    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    # stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * next_yr_EPS).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * next_yr_EPS).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    # stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_nextYr"] = round(next_yr_EPS, 2)
    # stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM # it means a comparison between equity return and 10 years risk free, usually ERP >= 3 for short term invest at least, ERP >= 5 for long term invest 

    # stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    stock_consolidate_df['FCF_per_share_TTM'] = round((qtr_cashflow_df['operatingCashflow'] - qtr_cashflow_df['capitalExpenditures']).tail(4).sum() / qtr_balancesheet_df['commonStockSharesOutstanding'].values[-1], 2)
    stock_consolidate_df['PFCF_TTM'] = round(stock_consolidate_df['stock_price'][0]  / stock_consolidate_df['FCF_per_share_TTM'], 2)
    stock_consolidate_df['FCF_yield_TTM'] = round((stock_consolidate_df['FCF_per_share_TTM'] / stock_consolidate_df['stock_price'][0]) * 100, 2) # % return on cash of every single dollar you spent on stock price, the higher the better

    # EPS trend for next year estimates
    stock_consolidate_df['next_yr_days7ago_EPS'] = round(next_yr_days7ago_EPS, 2)
    stock_consolidate_df['next_yr_days30ago_EPS'] = round(next_yr_days30ago_EPS, 2)
    stock_consolidate_df['next_yr_days60ago_EPS'] = round(next_yr_days60ago_EPS, 2)
    stock_consolidate_df['next_yr_days90ago_EPS'] = round(next_yr_days90ago_EPS, 2)

    # symbol growth rate
    stock_consolidate_df['curr_yr_growthrate_symbol'] = round(curr_yr_growthrate_symbol, 2)
    stock_consolidate_df['next_yr_growthrate_symbol'] = round(next_yr_growthrate_symbol, 2)
    stock_consolidate_df['curr_yr_growthrate_index'] = round(curr_yr_growthrate_index, 2)
    stock_consolidate_df['next_yr_growthrate_index'] = round(next_yr_growthrate_index, 2)

    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_nextYr"
    # ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    # ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"PFCF_TTM"
    ,"FCF_yield_TTM" 
    # ,"MA200_slope"
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        
        ticker_screen_df.loc[j] = stock_consolidate_df_values

# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 CL
1 ACN
2 NFLX
3 PG


In [17]:
ticker_dict_pd['NFLX']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_8yr_avg,PE_8yr_volatility_+,PE_8yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,EarningYield_TTM,ERP_TTM,BVPS_latest,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index
2026-01-23,NFLX,86.120,0.56,0.82116,2.53,2.69116,3.82,34.04,44.53,50.07,38.99,68.69,104.8,32.58,126.68,98.64,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,6.37,51.18,2.94,-1.32,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-22,NFLX,83.540,0.56,0.82116,2.53,2.69116,3.82,33.02,44.53,50.07,38.99,68.69,104.8,32.58,126.68,98.64,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,6.37,51.18,3.03,-1.23,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-21,NFLX,85.360,0.56,0.82116,2.53,2.69116,3.82,33.74,44.53,50.07,38.99,68.69,104.8,32.58,126.68,98.64,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,6.37,51.18,2.96,-1.30,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-20,NFLX,87.260,0.59,0.82116,2.40,2.69116,3.82,36.36,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.75,-1.51,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-16,NFLX,88.000,0.59,0.82116,2.40,2.69116,3.82,36.67,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.73,-1.53,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-15,NFLX,88.050,0.59,0.82116,2.40,2.69116,3.82,36.69,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.73,-1.53,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-14,NFLX,88.550,0.59,0.82116,2.40,2.69116,3.82,36.90,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.71,-1.55,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-13,NFLX,90.320,0.59,0.82116,2.40,2.69116,3.82,37.63,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.66,-1.60,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-12,NFLX,89.410,0.59,0.82116,2.40,2.69116,3.82,37.25,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.68,-1.58,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1
2026-01-09,NFLX,89.460,0.59,0.82116,2.40,2.69116,3.82,37.27,44.53,50.07,38.99,68.69,104.8,32.58,120.17,93.58,112.66,134.75,104.93,119.84,191.51,149.13,170.32,undervalued,12.13,59.37,2.68,-1.58,5.96,23.03,4.34,3.9,3.91,3.92,3.92,23.54,22.38,15.36,15.1


# Stock screener

In [18]:
ticker_screen_df = ticker_screen_df.sort_values(by=['price_valuation_assessment']
                                                ,ascending=False)

selected_cols = [
    'Ticker'

    ,'BVPS_latest'
    ,'stock_price'
    # Earnings
    ,'EPS_TTM'
    ,"EPS_nextYr"

    # PE relative valuation
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,'price_valuation_assessment'

    # Growth
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"

    # Other valuation metrics
    ,'ERP_TTM'
    # ,'latest_qtr_liquidation_mktcap_ratio_%'
    ,'PFCF_TTM'
    ,'FCF_yield_TTM'
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ,'Industry_PE_TTM_avg'
]

ticker_screen_df[selected_cols]

,Ticker,BVPS_latest,stock_price,EPS_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_7yr_avg,PE_7yr_volatility_+,PE_7yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,ERP_TTM,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index,Industry_PE_TTM_avg
26,PEP,14.13,139.91,7.85,8.56,17.82,18.51,23.86,27.05,20.67,149.46,141.14,145.3,142.03,134.12,138.07,163.07,153.99,158.53,undervalued,-4.98,9.11,1.42,21.52,4.65,8.57,8.57,8.57,8.51,-0.57,5.57,15.65,14.79,20.45
3,CPB,13.23,26.43,2.86,2.62,9.24,10.36,14.82,18.14,11.5,31.12,28.14,29.63,29.44,26.63,28.04,28.52,25.79,27.16,undervalued,-5.38,-8.36,6.63,11.49,8.7,2.63,2.63,2.67,2.65,-18.01,7.62,15.65,14.79,20.45
21,LW,12.57,41.37,3.4,3.03,12.17,16.93,25.85,35.12,16.58,64.97,50.15,57.56,56.37,43.51,49.94,57.93,44.71,51.32,undervalued,-13.25,-10.84,4.03,6.25,16.0,3.04,3.51,3.51,3.51,-18.32,10.79,15.65,14.79,20.45
20,KR,10.75,59.51,4.72,5.31,12.61,14.04,13.06,14.14,11.98,69.2,63.34,66.27,77.22,70.69,73.96,77.8,71.22,74.51,undervalued,11.6,12.44,3.74,17.35,5.76,5.3,5.29,5.27,5.27,7.37,10.58,15.65,14.79,20.45
19,KHC,34.95,23.42,2.76,2.51,8.49,8.9,12.19,13.4,10.98,25.14,23.98,24.56,22.91,21.85,22.38,22.88,21.82,22.35,undervalued,-8.9,-9.01,7.59,7.65,13.07,2.52,2.52,2.53,2.66,-17.42,-0.62,15.65,14.79,20.45
18,KMB,4.0,97.92,6.49,7.65,15.09,17.3,19.79,22.18,17.4,122.86,101.7,112.28,136.09,112.66,124.38,144.79,119.86,132.32,undervalued,10.77,17.86,2.44,20.27,4.93,7.66,7.64,7.66,7.67,2.41,2.31,15.65,14.79,20.45
28,PG,21.86,141.87,6.89,7.33,20.59,21.71,25.35,27.94,22.76,155.51,143.66,149.58,148.95,137.6,143.27,165.51,152.9,159.2,undervalued,-4.22,6.43,0.67,23.14,4.32,7.35,7.38,7.39,7.38,2.33,4.93,15.65,14.79,20.45
33,WMT,12.0,114.53,2.57,2.98,44.56,41.91,24.16,33.53,14.79,112.18,103.24,107.71,112.89,103.89,108.39,130.06,119.69,124.88,overvalued,0.64,15.94,-1.95,73.42,1.36,2.98,2.96,2.94,2.95,5.1,12.95,15.65,14.79,20.45
13,EL,10.71,113.73,1.68,2.93,67.7,59.96,45.66,60.85,30.47,108.11,93.36,100.74,115.69,99.91,107.8,188.72,162.97,175.84,overvalued,7.02,74.56,-2.71,39.49,2.53,2.93,2.92,2.92,2.85,43.21,35.61,15.65,14.79,20.45
1,ADM,46.48,61.94,3.69,4.24,16.79,15.96,13.97,17.85,10.09,60.11,57.67,58.89,55.69,53.44,54.56,69.0,66.21,67.6,overvalued,-7.35,14.8,1.77,6.37,15.69,4.28,4.35,4.38,4.65,-28.54,25.06,15.65,14.79,20.45


# Price EPS chart

In [11]:
# Parameters section
alpha_vantage_api_key = API_KEY


ticker_symbols = [
    'NFLX'
]



In [12]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])




    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year


            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue



    # Consolidated section
    stock_consolidate_df = Daily_stock_df
    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr


    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数


0 NFLX


In [13]:
import plotly.graph_objects as go


# Create the figure
fig = go.Figure()

# Add stock_price on primary y-axis (left)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["stock_price"],
    mode='lines',
    line=dict(color='black'),
    name='Stock Price',
    yaxis="y1"
))

# Add EPS_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["EPS_TTM"],
    mode='lines',
    fill='tonexty',  # Shadow effect
    line=dict(color='green'),
    name='EPS_TTM',
    yaxis="y2"
))

# Add PE_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["PE_TTM"],
    mode='lines',
    line=dict(color='blue', dash='dot'),
    name='PE_TTM',
    yaxis="y2"
))

# Update layout to remove grid and configure dual y-axes
fig.update_layout(
    title="Stock Metrics Over Time",
    xaxis=dict(title="Date", showgrid=False),
    yaxis=dict(
        title="Stock Price",
        showgrid=False,
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="EPS_TTM & PE_TTM",
        overlaying="y",  # Overlay with y1
        side="right",
        showgrid=False,
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue")
    ),
    legend_title="Metrics",
    template="plotly_white",
    plot_bgcolor='white'  # Background color
)

# Show the plot
fig.show()

# Financial statement of single stock

In [ ]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'PPG'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 15 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()

    
    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 PPG


In [ ]:
qtr_balancesheet_df.tail(10)

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,totalAssets_YoY,totalCurrentAssets_YoY,cashAndCashEquivalentsAtCarryingValue_YoY,cashAndShortTermInvestments_YoY,inventory_YoY,currentNetReceivables_YoY,totalNonCurrentAssets_YoY,propertyPlantEquipment_YoY,accumulatedDepreciationAmortizationPPE_YoY,intangibleAssets_YoY,intangibleAssetsExcludingGoodwill_YoY,goodwill_YoY,investments_YoY,longTermInvestments_YoY,shortTermInvestments_YoY,otherCurrentAssets_YoY,otherNonCurrentAssets_YoY,totalLiabilities_YoY,totalCurrentLiabilities_YoY,currentAccountsPayable_YoY,deferredRevenue_YoY,currentDebt_YoY,shortTermDebt_YoY,totalNonCurrentLiabilities_YoY,capitalLeaseObligations_YoY,longTermDebt_YoY,currentLongTermDebt_YoY,longTermDebtNoncurrent_YoY,shortLongTermDebtTotal_YoY,otherCurrentLiabilities_YoY,otherNonCurrentLiabilities_YoY,totalShareholderEquity_YoY,treasuryStock_YoY,retainedEarnings_YoY,commonStock_YoY,commonStockSharesOutstanding_YoY,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio
9,2023-06-30,21614000000,8068000000,1228000000,1228000000,2506000000,-1296000000,13546000000,3.418000e+09,NaN,2493000000,2493000000,6294000000,NaN,262000000.0,68000000.0,5562000000,NaN,5416000000,5416000000,4.306000e+09,NaN,NaN,997000000,9008000000,188000000.0,6.099000e+09,8.090000e+08,NaN,7096000000,4419000000,2.332000e+09,7614000000,-1.351200e+10,21290000000,969000000,237300000,-2.194669,-1.525693,-13.884993,-13.884993,-3.578299,-136.050070,-2.588811,1.877794,0.0,2.088452,2.088452,2.125588,NaN,2.745098,21.428571,975.822050,NaN,-63.518793,11.257190,5.126953,NaN,NaN,153.045685,-9.721387,1.621622,-13.880260,287.081340,NaN,-12.481500,-1.229325,0.301075,6.608793,-0.022198,1.642318,0.0,0.168848,1.489660,2652000000,2.299774,0.711321,1.026957
8,2023-09-30,21501000000,7552000000,1218000000,1218000000,2217000000,3612000000,13949000000,4.231000e+09,NaN,2416000000,2416000000,6181000000,NaN,264000000.0,64000000.0,441000000,NaN,13566000000,5100000000,4.210000e+09,NaN,NaN,795000000,8466000000,811000000.0,5.596000e+09,6.060000e+08,NaN,7013000000,4305000000,6.520000e+08,7751000000,NaN,21563000000,969000000,237500000,-0.522809,-6.395637,-0.814332,-0.814332,-11.532322,-378.703704,2.975048,23.785840,0.0,-3.088648,-3.088648,-1.795361,NaN,0.763359,-5.882353,-92.071197,NaN,150.480059,-5.834564,-2.229447,NaN,NaN,-20.260782,-6.016874,331.382979,-8.247254,-25.092707,NaN,-1.169673,-2.579769,-72.041166,1.799317,0.000000,1.282292,0.0,0.084282,1.480784,2452000000,2.282219,1.750226,1.046078
7,2023-12-31,21647000000,7431000000,1493000000,1493000000,1934000000,3007000000,14216000000,NaN,NaN,2261000000,2261000000,6115000000,NaN,259000000.0,75000000.0,922000000,NaN,13624000000,5054000000,3.307000e+09,NaN,NaN,434000000,8570000000,816000000.0,5.748000e+09,3.060000e+08,NaN,6599000000,1185000000,6.540000e+08,7832000000,NaN,21500000000,969000000,237300000,0.679038,-1.602225,22.577997,22.577997,-12.764998,-16.749723,1.914116,0.000000,0.0,-6.415563,-6.415563,-1.067788,NaN,-1.893939,17.187500,109.070295,NaN,0.427539,-0.901961,-21.448931,NaN,NaN,-45.408805,1.228443,0.616523,2.716226,-49.504950,NaN,-5.903322,-72.473868,0.306748,1.045026,0.000000,-0.292167,0.0,-0.084211,1.470321,2377000000,2.418174,1.739530,1.087653
6,2024-03-31,21867000000,7671000000,1

In [ ]:
qtr_income_df.tail(20)

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin_%,operating_margin_%,net_margin_%
19,2020-12-31,1617000000,3757000000,2140000000,2140000000,510000000,882000000,100000000,1107000000,NaN,-26000000.0,5000000.0,31000000,NaN,5000000.0,NaN,125000000,345000000,67000000,NaN,136000000.0,NaN,376000000,501000000,272000000,-2.531646,1.953867,5.626851,5.626851,-14.858097,5.502392,8.695652,4.433962,NaN,-13.333333,-83.333333,-8.823529,NaN,25.000000,NaN,-5.303030,-39.685315,-45.967742,NaN,-69.642857,NaN,-37.953795,-32.113821,-38.461538,43.039659,13.574661,7.239819
18,2021-03-31,1649000000,3881000000,2232000000,2232000000,527000000,891000000,102000000,1122000000,NaN,-24000000.0,6000000.0,30000000,NaN,6000000.0,NaN,129000000,499000000,114000000,NaN,385000000.0,NaN,529000000,658000000,378000000,1.978973,3.300506,4.299065,4.299065,3.333333,1.020408,2.000000,1.355014,NaN,-7.692308,20.000000,-3.225806,NaN,20.000000,NaN,3.200000,44.637681,70.149254,NaN,183.088235,NaN,40.691489,31.337325,38.970588,42.489049,13.578974,9.739758
17,2021-06-30,1730000000,4359000000,2629000000,2629000000,531000000,955000000,107000000,1199000000,NaN,-25000000.0,6000000.0,31000000,NaN,6000000.0,NaN,137000000,593000000,160000000,NaN,433000000.0,NaN,624000000,761000000,431000000,4.912068,12.316413,17.786738,17.786738,0.759013,7.182941,4.901961,6.862745,NaN,4.166667,0.000000,3.333333,NaN,0.000000,NaN,6.201550,18.837675,40.350877,NaN,12.467532,NaN,17.958412,15.653495,14.021164,39.688002,12.181693,9.887589
16,2021-09-30,1639000000,4372000000,2733000000,2733000000,429000000,950000000,114000000,1210000000,NaN,-23000000.0,7000000.0,30000000,NaN,7000000.0,NaN,146000000,441000000,96000000,NaN,345000000.0,NaN,471000000,617000000,344000000,-5.260116,0.298234,3.955877,3.955877,-19.209040,-0.523560,6.542056,0.917431,NaN,-8.000000,16.666667,-3.225806,NaN,16.666667,NaN,6.569343,-25.632378,-40.000000,NaN,-20.323326,NaN,-24.519231,-18.922470,-20.185615,37.488564,9.812443,7.868253
15,2021-12-31,1498000000,4190000000,2692000000,2692000000,199000000,1034000000,116000000,1299000000,NaN,-23000000.0,23000000.0,30000000,NaN,7000000.0,NaN,149000000,282000000,4000000,NaN,278000000.0,NaN,312000000,461000000,286000000,-8.602807,-4.162855,-1.500183,-1.500183,-53.613054,8.842105,1.754386,7.355372,NaN,0.000000,228.571429,0.000000,NaN,0.000000,NaN,2.054795,-36.054422,-95.833333,NaN,-19.420290,NaN,-33.757962,-25.283630,-16.860465,35.751790,4.749403,6.825776
14,2022-03-31,1610000000,4308000000,2698000000,2698000000,376000000,974000000,115000000,1234000000,NaN,-21000000.0,9000000.0,30000000,NaN,9000000.0,NaN,145000000,78000000,55000000,NaN,23000000.0,NaN,108000000,253000000,18000000,7.476636,2.816229,0.222883,0.222883,88.944724,-5.802708,-0.862069,-5.003849,NaN,-8.695652,-60.869565,0.000000,NaN,28.571429,NaN,-2.684564,-72.340426,1275.000000,NaN,-91.726619,NaN,-65.384615,-45.119306,-93.706294,37.372331,8.727948,0.417827
13,2022-06-30,1737000000,4691000000,2954000000,2954000000,499000000,982000000,115000000,1238000000,NaN,-27000000.0,11000000.0,38000000,NaN

In [ ]:
pd.DataFrame(stock_financial_dict['PPG']['qtr']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%
19,2020-12-31,43.039659,13.574661,7.239819,278000000,1.411545,1982000000,2.608981,2.415363,1.051287,5689000000,966000000,134000000,-1256000000,46000000,4.781157,3464000000,15.79,-4.399551,-16.489776,-39.640875
18,2021-03-31,42.489049,13.578974,9.739758,385000000,1.433398,2219000000,2.404687,2.426785,1.059570,5839000000,-23000000,80000000,-424000000,469000000,6.473711,3668000000,16.72,-1.279309,0.031779,34.530405
17,2021-06-30,39.688002,12.181693,9.887589,433000000,1.356778,1958000000,3.262002,2.553989,0.951166,6242000000,604000000,62000000,-1818000000,585000000,6.904838,3387000000,15.44,-6.592398,-10.290037,1.517811
16,2021-09-30,37.488564,9.812443,7.868253,345000000,1.336262,1837000000,3.316277,2.474492,0.924584,6253000000,525000000,78000000,-34000000,-448000000,5.501359,3568000000,16.27,-5.541822,-19.449269,-20.422940
15,2021-12-31,35.751790,4.749403,6.825776,278000000,1.420443,1995000000,3.294236,2.376710,0.962908,6286000000,456000000,151000000,-128000000,-513000000,4.549793,3628000000,16.54,-4.632809,-51.598155,-13.249154
14,2022-03-31,37.372331,8.727948,0.417827,23000000,1.474671,2455000000,2.783707,2.501608,1.003094,6218000000,-304000000,194000000,-183000000,443000000,0.289482,3743000000,17.06,4.532754,83.769357,-93.878684
13,2022-06-30,37.028352,10.637391,9.400981,448000000,1.501646,2591000000,2.611347,2.471854,1.021297,6182000000,168000000,70000000,33000000,-230000000,7.133614,3842000000,17.52,-0.920409,21.877339,2149.968024
12,2022-09-30,36.862131,10.541629,7.363474,339000000,1.520503,2564000000,2.526521,2.440172,1.031060,6034000000,512000000,104000000,-96000000,-269000000,5.452436,3792000000,17.29,-0.448903,-0.900234,-21.673346
11,2022-12-31,37.323775,8.745520,5.686977,246000000,1.519381,2452000000,2.652121,2.129096,1.038128,6592000000,587000000,150000000,-215000000,-353000000,3.610437,4295000000,19.58,1.252355,-17.038255,-22.767737
10,2023-03-31,40.730594,8.333333,6.027397,273000000,1.683032,3325000000,2.129925,2.078689,1.149137,7142000000,85000000,120000000,-64000000,277000000,3.696444,4811000000,21.93,9.127743,-4.713115,5.985956


In [ ]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [ ]:
YoY_EPS_screener

,fiscalDateEnding,PPG_YoY_reportedEPS_%,PPG_reportedEPS
19,2005,NaN,NaN
18,2006,NaN,NaN
17,2007,NaN,NaN
16,2008,NaN,NaN
15,2009,NaN,NaN
14,2010,NaN,2.60
13,2011,30.384615,3.39
12,2012,17.404130,3.98
11,2013,4.271357,4.15
10,2014,17.590361,4.88


In [ ]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,PPG_PE
0,2024,14.75
1,2023,19.50
2,2022,20.75
3,2021,25.47
4,2020,25.30
5,2019,21.50
6,2018,17.36
7,2017,19.83
8,2016,16.03
9,2015,17.34


# SP500 sectors

## SP500 sector ticker

In [ ]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
# sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

NameError: name 'pd' is not defined

In [ ]:
# Pivot the DataFrame
sp500_pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
sp500_pivot_sector_ticker_df = sp500_pivot_sector_ticker_df.set_index('Sector').T

sp500_pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBY, BKNG, BWA, CZR, K...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, APO, ACGL, AJG,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, TECH, BII...","[MMM, AOS, ALLE, AME, ADP, AXON, BA, BR, BLDR,...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [ ]:
sp500_pivot_sector_ticker_df['Consumer Staples'].iloc[0]

['MO',
 'ADM',
 'BF.B',
 'BG',
 'CPB',
 'CHD',
 'CLX',
 'KO',
 'CL',
 'CAG',
 'STZ',
 'COST',
 'DG',
 'DLTR',
 'EL',
 'GIS',
 'HSY',
 'HRL',
 'K',
 'KVUE',
 'KDP',
 'KMB',
 'KHC',
 'KR',
 'LW',
 'MKC',
 'TAP',
 'MDLZ',
 'MNST',
 'PEP',
 'PM',
 'PG',
 'SJM',
 'SYY',
 'TGT',
 'TSN',
 'WBA',
 'WMT']

In [ ]:
len(sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

50

## SP 500 earnings by years

In [ ]:
sp500_eps_df= pd.read_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est.xlsx'
                            ,sheet_name='QUARTERLY DATA')

In [ ]:
sp500_eps_df.columns = [
    'Quarter'
    ,'Ops EPS(Opearting)'
    ,'Reported EPS(GAAP)'
    ,'Div/share'
    ,'Sales/share'
    ,'BVPS'
    ,'CapEx/share'
    ,'Price'
    ,'Divisor'
]

In [ ]:
sp500_eps_df = sp500_eps_df[5::].fillna(0)
sp500_eps_df_no_first = sp500_eps_df \
    .iloc[1:] \
    .sort_values(['Quarter'], ascending=True) \
    .copy()

#####################
# Column enrichment #
#####################
sp500_eps_df['Year'] = pd.to_datetime(sp500_eps_df['Quarter']).dt.year
sp500_eps_df["Month"] = pd.to_datetime(sp500_eps_df['Quarter']).dt.month

# Calculate year end EPS
last_quarter_prices_df = sp500_eps_df[sp500_eps_df["Month"] == 12][["Year", "Price"]]
price_map = dict(zip(last_quarter_prices_df["Year"], last_quarter_prices_df["Price"]))
sp500_eps_df["Yearly price"] = sp500_eps_df["Year"].map(price_map)


# Calculate rolling 4-quarter sum
sp500_eps_df_no_first["Ops EPS(Opearting) TTM"] = sp500_eps_df_no_first["Ops EPS(Opearting)"].rolling(window=4).sum()
sp500_eps_df_no_first["Reported EPS(GAAP) TTM"] = sp500_eps_df_no_first["Reported EPS(GAAP)"].rolling(window=4).sum()
sp500_eps_df_no_first["Div/share TTM"] = sp500_eps_df_no_first["Div/share"].rolling(window=4).sum()


# Merge back with original dataframe
sp500_eps_df = sp500_eps_df.merge(
    sp500_eps_df_no_first[
        [
            "Quarter"
            ,"Ops EPS(Opearting) TTM"
            ,'Reported EPS(GAAP) TTM'
            ,'Div/share TTM'
            ]
        ]
    ,on="Quarter"
    ,how="left"
    )


######################
# Metrics calcuation #
######################
sp500_eps_df['PE TTM Ops'] = sp500_eps_df['Price'] / sp500_eps_df['Ops EPS(Opearting) TTM']
sp500_eps_df['PE TTM GAAP'] = sp500_eps_df['Price'] / sp500_eps_df['Reported EPS(GAAP) TTM']
sp500_eps_df['Earnings yield(Ops)'] = round((sp500_eps_df['Ops EPS(Opearting) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Earnings yield(GAAP)'] = round((sp500_eps_df['Reported EPS(GAAP) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Div payout ratio(Opearting)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Ops EPS(Opearting) TTM']) * 100 ,2)
sp500_eps_df['Div payout ratio(GAAP)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Reported EPS(GAAP) TTM']) * 100, 2)
# sp500_eps_df["Ops EPS(Opearting) TTM Growth Rate"] = round(((sp500_eps_df["Ops EPS(Opearting) TTM"] - sp500_eps_df["Ops EPS(Opearting) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Ops EPS(Opearting) TTM"])*100, 2)
# sp500_eps_df["Reported EPS(GAAP) TTM Growth Rate"] = round(((sp500_eps_df["Reported EPS(GAAP) TTM"] - sp500_eps_df["Reported EPS(GAAP) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Reported EPS(GAAP) TTM"])*100, 2)


# 1.section: YoY growth rate
sp500_eps_df_yrly_ops_eps = sp500_eps_df.groupby(['Year'])['Ops EPS(Opearting)'].sum().reset_index()
sp500_eps_df_yrly_gaap_eps = sp500_eps_df.groupby(['Year'])['Reported EPS(GAAP)'].sum().reset_index()
sp500_eps_df_yrly_ops_eps['YoY growth rate(Ops)'] = sp500_eps_df_yrly_ops_eps['Ops EPS(Opearting)'].pct_change()
sp500_eps_df_yrly_gaap_eps['YoY growth rate(GAAP)'] = sp500_eps_df_yrly_gaap_eps['Reported EPS(GAAP)'].pct_change()

# Create mapping dictionary: {year: YoY growth}
growth_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["YoY growth rate(Ops)"]))
growth_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["YoY growth rate(GAAP)"]))
yrly_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["Ops EPS(Opearting)"]))
yrly_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["Reported EPS(GAAP)"]))

# Map back to original df
sp500_eps_df["YoY growth rate(Ops)"] = round(sp500_eps_df["Year"].map(growth_map_ops_eps)*100, 2)
sp500_eps_df["YoY growth rate(GAAP)"] = round(sp500_eps_df["Year"].map(growth_map_gaap_eps)*100, 2)
sp500_eps_df["Yearly EPS (Ops)"] = sp500_eps_df["Year"].map(yrly_map_ops_eps)
sp500_eps_df["Yearly EPS (GAAP)"] = sp500_eps_df["Year"].map(yrly_map_gaap_eps)


# 2.section: PE TTM AVG Section
PE_TTM_ops_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM Ops"] != 0, "PE TTM Ops"].mean(skipna=True)
PE_TTM_gaap_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM GAAP"] != 0, "PE TTM GAAP"].mean(skipna=True)
sp500_eps_df['PE TTM Ops AVG'] = round(PE_TTM_ops_avg, 2)
sp500_eps_df['PE TTM GAAP AVG'] = round(PE_TTM_gaap_avg, 2)

# 3. section: PE 5 years rolling average
sp500_eps_df['PE Yrly Ops'] = sp500_eps_df['Yearly price'] / sp500_eps_df["Yearly EPS (Ops)"]
sp500_eps_df['PE Yrly GAAP'] = sp500_eps_df["Yearly price"] / sp500_eps_df["Yearly EPS (GAAP)"]

sp500_eps_df_yrly_ops_PE = sp500_eps_df[sp500_eps_df["PE Yrly Ops"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly Ops"] \
                            .mean()
sp500_eps_df_yrly_gaap_PE = sp500_eps_df[sp500_eps_df["PE Yrly GAAP"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly GAAP"] \
                            .mean()

sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=10, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=10, min_periods=1).mean()


# Create mapping dictionaries
PE5_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"]))
PE10_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"]))
PE5_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"]))
PE10_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"]))


# Map to quarterly dataframe
sp500_eps_df["PE Ops 5Y Avg"] = sp500_eps_df["Year"].map(PE5_ops_map)
sp500_eps_df["PE Ops 10Y Avg"] = sp500_eps_df["Year"].map(PE10_ops_map)
sp500_eps_df["PE GAAP 5Y Avg"] = sp500_eps_df["Year"].map(PE5_gaap_map)
sp500_eps_df["PE GAAP 10Y Avg"] = sp500_eps_df["Year"].map(PE10_gaap_map)

In [ ]:
sp500_eps_df.head(10)

,Quarter,Ops EPS(Opearting),Reported EPS(GAAP),Div/share,Sales/share,BVPS,CapEx/share,Price,Divisor,Year,Month,Yearly price,Ops EPS(Opearting) TTM,Reported EPS(GAAP) TTM,Div/share TTM,PE TTM Ops,PE TTM GAAP,Earnings yield(Ops),Earnings yield(GAAP),Div payout ratio(Opearting),Div payout ratio(GAAP),YoY growth rate(Ops),YoY growth rate(GAAP),Yearly EPS (Ops),Yearly EPS (GAAP),PE TTM Ops AVG,PE TTM GAAP AVG,PE Yrly Ops,PE Yrly GAAP,PE Ops 5Y Avg,PE Ops 10Y Avg,PE GAAP 5Y Avg,PE GAAP 10Y Avg
0,2025-06-30,0.00,0.00,19.484935,0.000000,0.000000,0.000000,6204.953952,8461.081057,2025,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
1,2025-03-31,57.51,53.89,19.372895,488.420003,1197.110200,30.911700,5611.852607,8473.831556,2025,3,NaN,236.24,216.69,76.145301,23.754879,25.898069,4.21,3.86,32.23,35.14,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
2,2024-12-31,61.21,57.69,19.810500,508.588989,1178.567800,34.471600,5881.627648,8467.894518,2024,12,5881.627648,233.36,210.17,74.832255,25.204095,27.985096,3.97,3.57,32.07,35.61,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
3,2024-09-30,59.16,51.99,18.680400,501.354000,1166.346100,30.891700,5762.484883,8451.342154,2024,9,5881.627648,226.05,200.27,73.400300,25.492081,28.773580,3.92,3.48,32.47,36.65,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
4,2024-06-28,58.36,53.12,18.281506,488.705399,1139.314000,28.457100,5460.482621,8395.388376,2024,6,5881.627648,219.14,195.93,71.975800,24.917781,27.869559,4.01,3.59,32.84,36.74,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
5,2024-03-28,54.63,47.37,18.059849,471.949169,1116.727900,27.060700,5254.354401,8388.772132,2024,3,5881.627648,215.62,191.39,70.824826,24.368585,27.453652,4.10,3.64,32.85,37.01,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
6,2023-12-29,53.90,47.79,18.378545,490.201506,1106.214712,30.133461,4769.829411,8394.128747,2023,12,4769.829411,213.53,192.43,70.303692,22.337983,24.787348,4.48,4.03,32.92,36.53,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
7,2023-09-29,52.25,47.65,17.255900,468.524082,1076.237542,27.013050,4288.054123,8380.848066,2023,9,4769.829411,210.00,184.25,69.313136,20.419305,23.273021,4.90,4.30,33.01,37.62,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
8,2023-06-30,54.84,48.58,17.130532,462.132679,1061.913100,25.961900,4450.381312,8350.369620,2023,6,4769.829411,208.10,181.01,68.714936,21.385782,24.586384,4.68,4.07,33.02,37.96,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
9,2023-03-31,52.54,48.41,17.538715,451.437952,1045.266400,25.460100,4109.312445,8357.127040,2023,3,4769.829411,200.13,175.17,68.211442,20.533216,23.458997,4.87,4.26,34.08,38.94,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973


In [ ]:
sp500_eps_df.to_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est-hist.xlsx')

# Stock PE by years by tickers

In [ ]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value
stock_PE_annual_consolidate_df = pd.DataFrame()

ticker_symbols = [
'MNST'
,'KO'
,'KDP'
,'PEP'
 ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 10 # test for 'X' years PE 

In [ ]:

# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()


    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values


    ####################
    ### Consolidated ###
    ####################

    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)


stock_PE_annual_consolidate_df['industry_avg_PE'] = stock_PE_annual_consolidate_df.iloc[:, 1:].sum(axis=1) / (stock_PE_annual_consolidate_df.shape[1] -1)  # Sum by row

0 MNST
1 KO
2 KDP
3 PEP


In [ ]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,MNST_PE,KO_PE,KDP_PE,PEP_PE,industry_avg_PE
0,2024,43.44,26.72,23.97,24.53,29.6650
1,2023,36.46,21.91,18.61,22.29,24.8175
2,2022,22.76,25.65,21.23,26.61,24.0625
3,2021,18.68,25.41,23.04,27.79,23.7300
4,2020,17.25,28.12,22.86,26.87,23.7750
5,2019,15.35,26.23,23.73,24.76,22.5175
6,2018,13.67,22.66,8.90,19.55,16.1950
7,2017,22.77,24.02,22.11,23.20,23.0250
8,2016,17.05,21.71,20.61,29.98,22.3375
9,2015,25.08,21.48,23.18,21.91,22.9125


## Appendix

In [ ]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    'KR', # DATA ISSUE
    'LW',
    'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO', # PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [ ]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]